### Enxi Cui 17210995

### Imports

In [17]:
import math
import unittest
from DataStructures import create_airport_list, createAircrafts, Graph
import pandas as pd

###  Correctness, Speed, Efficiency, Security, Robustness, Clarity, Maintainability
- **correctness**: by using the unit tests to test **correctness** of the hash table
- **speed and efficiency**: by using the %%time to calculate the **speed** of running these methods, and to show the **efficiency**
- **Security**, name mangling was used to make the variables "private".
- **Robustness** Unit testing to check to robustness
- **clarity**: the comments to the code help with the understanding and the **clarity** of these methods, as does the breaking down of the code into many methods.
- **maintainability**: the code has been broken into small segments, which is easy to read, change, and modify, and code has been commented.


### Design
**Scalability** & **Flexibility**

The hash table has 5832//4 lists, as it fits with the assigned problem well, and anything larger would use extra memory for no benefit to this program. This hash table could be used for other assignments which use an equal or lower amount of data. 

The reason that I used separate chaining, not a linear probing, is because when deleting, it cannot be left null, and it needs to rehash the cluster to the right of the deleted key.
      


### Code
- OOP principle: encapsulation(Getter and Setter)
- Clarity: every fuction has comments, and the variable names are easy to understand
- Error handling: in the delete part, I add an if not self.isEmpty(), so if it is empty, it will not implement


I created a separate chaining hash table to put the different airports objects into it, this allows us to easily access the airports for calculating the distance between them, the hash table have 5832//4 lists, the reason why it divided by 4 is just make sure each index of the chain is not too long or too empty

### My Hash Table Class

In [2]:

class hashTable:
#   create a hash table which has 5832//4 lists with empty data and the default size is 0.
    def __init__(self):
        self.__hash_table = [[] for _ in range(5832//4)]
        self.__size = 0
        self.__key = []
        
#   inserting key-value pairs into the hash table “O（n）” 
    def insert(self, key, value):
        #  Determine which hash key should be used
        hash_key = hash(key) % len(self.__hash_table)
        key_exists = False
        bucket = self.__hash_table[hash_key]
#  If the key is already present in the hash table, then update its value with the new value.
#  Otherwise, insert new key-value pairs into the hash table, and size plus one
        for i, kv in enumerate(bucket):
            k, v = kv
            if key == k:
                key_exists = True 
                break
        if key_exists:
            bucket[i] = ((key, value))
        else:
            bucket.append((key, value)) 
            self.__key += key
            self.__size +=1
        
# search if the keys are in the hash table, if it is present, return the value   O(n)
    def search(self, key):
        hash_key = hash(key) % len(self.__hash_table)    
        bucket = self.__hash_table[hash_key]
        for i, kv in enumerate(bucket):
            k, v = kv
            if key == k:
                return v
            
# use to delete keys     O(n)       
    def delete(self, key):
        if not self.isEmpty():
            hash_key = hash(key) % len(self.__hash_table)    
            key_exists = False
            bucket = self.__hash_table[hash_key]
    # If the key is already in the hash table, then delete that particular key-value pair from the hash 
    # table, and size minus 1.
    # –Otherwise, no operation is done. 
            for i, kv in enumerate(bucket):
                k, v = kv 
                if key == k: 
                    key_exists = True 
                    break
            if key_exists:
                del bucket[i]         
                self.__size -=1
                self.__key.remove(key)
        
# check is the key and value in the hash table  "O(n)"        
    def contains(self, key):
        hash_key = hash(key) % len(self.__hash_table)
        bucket = self.__hash_table[hash_key]
# If the key is already in the hash table, then return true. Otherwise, return false
        for i, kv in enumerate(bucket):
            k, v = kv
            if key ==k:
                 return True
        return False
    
#  O(n^2)   
    def keys(self):
        """
        key = []
        for i in self.hash_table:
            if len(i) != 0:
                for j in i:
                    key.append(j[0])         
        """
        return self.__key
# """
# originally, keys() used a nested for loop to add the keys, we can now do this in O(1)
# instead of O(n^2)
# """     
    
# to check if the hash table is empty   O(1)   
    def isEmpty(self):
        return self.__size == 0

# """
# originally, isEmpty used the for loop to check if it's empty. By optimizing getSize, we can now do this in O(1)
# instead of O(n)
# """  

# O(1)    
    def getSize(self):   
        return self.__size
# """
# originally, getSize use the nested for loop to return the number of keys, we optimized this by storing the 
# size directly, increasing it and decrease it when we need it to, this means the getSize is now O(1) instead of O(n^2)
# """

### Read the Files

In [4]:
test_df = pd.read_csv("test.csv")
airport_df = pd.read_csv("airport.csv")
country_currency_df = pd.read_csv("countrycurrency.csv")
currency_rates_df = pd.read_csv("currencyrates.csv")
aircraft_df = pd.read_csv("aircraft.csv")
airport_currencies_df = airport_df.merge(country_currency_df.rename(columns={"name":"Country"}), on="Country")
merged_df = airport_currencies_df.merge(currency_rates_df.rename(columns={"CurrencyCode":"currency_alphabetic_code"}), on="currency_alphabetic_code")
airports = create_airport_list(merged_df)
aircrafts = createAircrafts(aircraft_df)

In [5]:
# get the row in the dataframe, with airplanes removed    O(1)
def getRoute(test_df, row):  
    route = list(test_df.drop("Airplane", axis=1).iloc[row])
    return route

# add the airports in the route to the graph if it is not there  O(n)
def addToAirportAtlas(route, g, airports):
    for ap in route:
        if not g.containsVertex(ap):
            g.insertVertex(ap, airports.search(ap).get_conversion_rate())

#add edges between airports to the g raph if not it is not there       O(n^2)   
def addAirportDistances(route, g, airports):
    for ap in route:
        for ap2 in route:
            if not g.containsEdge(ap, ap2) and not g.containsEdge(ap2, ap):
                g.insertEdge(ap, ap2, airports.search(ap).get_distance(airports.search(ap2)))


### Unit Test

In [6]:
class unit_tests(unittest.TestCase):
  
    def test_insert(self):
        t = hashTable()
        t.insert("3", 3)
        self.assertEqual(t.search("3"), 3)  
    
    def test_delete(self):
        t = hashTable()
        t.insert('1', 3)
        t.insert('2', 3)
        t.insert('3', 3)
        t.delete('2')
        self.assertEqual(t.search("2"), None)

    def test_contains(self):
        t = hashTable()
        self.assertEqual(t.contains("2"), False)
        
    def test_keys(self):
        t = hashTable()
        t.insert('1', 3)
        t.insert('2', 3)
        t.insert('3', 3)
        self.assertEqual(t.keys(),["1", "2", "3"])
        
    def test_isEmpty(self):
        t = hashTable()
        self.assertTrue(t.isEmpty())
        
    def test_size(self): 
        t = hashTable()
        t.insert('1', 3)
        t.insert('2', 3)
        t.insert('3', 3)
        self.assertEqual(t.getSize(),3)
        
    def test_getRoute(self):
        self.assertEqual(getRoute(test_df,1), ["SNN", "ORK", "MAN", "CDG", "SIN"])
                         
    def test_addToAirportAtlas(self):
        G = Graph()
        getroute = getRoute(test_df,1)
        addToAirportAtlas(getroute, G, airports)
        self.assertEqual(G.getVertices(), ["SNN", "ORK", "MAN", "CDG", "SIN"])
        
    def test_addAirportDistances(self):
        G = Graph()
        getroute = getRoute(test_df,1)
        addToAirportAtlas(getroute, G, airports)
        addAirportDistances(getroute, G, airports)
        self.assertEqual(G.getEdgeCount(), 10)
        
        
unittest.main(argv=[''], verbosity=2, exit=False)


test_addAirportDistances (__main__.unit_tests) ... ok
test_addToAirportAtlas (__main__.unit_tests) ... ok
test_contains (__main__.unit_tests) ... ok
test_delete (__main__.unit_tests) ... ok
test_getRoute (__main__.unit_tests) ... ok
test_insert (__main__.unit_tests) ... ok
test_isEmpty (__main__.unit_tests) ... ok
test_keys (__main__.unit_tests) ... ok
test_size (__main__.unit_tests) ... ok

----------------------------------------------------------------------
Ran 9 tests in 0.014s

OK


'\n    def test_init(self):\n        self.hash_table = [[] for _ in range(10)]\n        hashTable = hash_table()\n        d = Dict(a=1, b=\'test\')\n        self.assertEquals(d.a, 1)\n        self.assertEquals(d.b, \'test\')\n        self.assertTrue(isinstance(d, dict)\n# myAir = Aircraft(\'238\', "imperial", 1000)\n# print(myAir)\n\n    \n# print("Departing at:", lhr.get_name()\n# print("Destination:", dub.get_name())\n# print("Distance:", lhr.get_distance(dub))\n# '

In [7]:
t = hashTable()
t.insert('1', 3)
t.insert('2', 3)
t.insert('3', 3)


In [8]:
%time
t.insert('1', 3)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 6.2 µs


In [9]:
%time
t.search('1')

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.25 µs


3

In [10]:
%time
t.contains('1')

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 6.91 µs


True

In [11]:
%time
t.keys()

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 7.15 µs


['1', '2', '3']

In [12]:
%time
t.isEmpty()

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 4.77 µs


False

In [13]:
%time
t.getSize()

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 6.2 µs


3

# Compare

In [14]:
%%time
d = {}

for i in range (10000):
    d[i] = i

CPU times: user 1.62 ms, sys: 735 µs, total: 2.35 ms
Wall time: 2.37 ms


In [15]:
%%time
t = hashTable()
for i in range (10000):
    key = str(i)
    t.insert(key, i)

CPU times: user 31.3 ms, sys: 3.35 ms, total: 34.6 ms
Wall time: 35.7 ms


By comparing the hash table I created with Python's inbuilt dictionary, we can see that a dictionary takes less time to add values and keys than a hash table, and as the number of keys and values are created increases, the time is slower, and when the number is large enough, the hash table may take too long to output. 

I think the main reason why it is much slower than a dictionary is creating a dictionary is Python's built in function, so it must be highly-optimized. 

But the reason why that I still want to use the hash table is even though it is slower than a dictionary, it still quite fast. In addition, we learned dictionaries last semester, we are familiar with it, and in this module: Data structure and Algorithm, we were introduced to this data structure. Therefore, I wanted to gain a better understanding of how it worked and why it was useful.

# Summary

#### By doing this project, I learned how to create a hash table by using python. I put the Correctness, Speed, Efficiency, Security, Robustness, Clarity, Maintainability of the hash table into consideration. By using unit tests to test correctness, and calculate each function's time, and compare it with Python's inbuilt dictionary. I also tried to figure out how to optimize these methods. I optimized the size, isempty and keys to save CPU time and run faster. The optimized compexity makes the code more efficient. Name mangling was used to make the variables "private" to keep it secure. I added comments to the code to help with the understanding and clarity of my work. The code has be broken into small segments, which is easy to read, change and modify. 

#### Actually, the most difficult thing for me was to create a mindset that breaks down big problems into small ones, because when I just got this project, I was not quite sure what thing should I do first, and thanks to my teammates who helped assign each person's tasks and build a framework, I was able to understand what was required of me, how to go about finishing my part, and how it fit in with the rest of this project.